In [1]:
from __future__ import division
from __future__ import print_function

import time
import os

# Train on CPU (hide GPU) due to memory constraints
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
import scipy
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from gae.optimizer import OptimizerAE, OptimizerVAE
from gae.input_data import load_data, parse_index_file
from gae.model import GCNModelAE, GCNModelVAE
from gae.preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, mask_test_edges

In [2]:
# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 16, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0., 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')

flags.DEFINE_string('model', 'gcn_ae', 'Model string.')
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')
flags.DEFINE_integer('features', 1, 'Whether to use features (1) or not (0).')

model_str = FLAGS.model
dataset_str = FLAGS.dataset

In [3]:
X = pd.read_hdf('history_small.hdf', key='hist') #_small
A = pd.read_pickle('adjacency_small.pkl') #_small
A = A[sorted(A.columns)]

In [4]:
X[X==1] = 2 #10 #valid
X[X==0] = 1 #not valid
X[X==-100] = 0 #missing
X = X.astype(float)
np.unique(X)

array([ 0.,  1.,  2.])

In [5]:
A.mean().mean()

0.7434050630239977

In [6]:
A[A<=0.7] = 0

In [7]:
adj = scipy.sparse.csr_matrix(A.values)
features = scipy.sparse.csr_matrix(X)#np.asmatrix(X)

In [8]:
print(features.shape, adj.shape)
print(type(features), type(adj)) 

(878, 290) (878, 878)
<class 'scipy.sparse.csr.csr_matrix'> <class 'scipy.sparse.csr.csr_matrix'>


In [9]:
adj

<878x878 sparse matrix of type '<class 'numpy.float64'>'
	with 516510 stored elements in Compressed Sparse Row format>

In [10]:
features

<878x290 sparse matrix of type '<class 'numpy.float64'>'
	with 8821 stored elements in Compressed Sparse Row format>

In [11]:
# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

In [12]:
adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj.eliminate_zeros()
# Check that diag is zero:
assert np.diag(adj.todense()).sum() == 0

adj_triu = sp.triu(adj)
adj_tuple = sparse_to_tuple(adj_triu)
edges = adj_tuple[0]
edges_all = sparse_to_tuple(adj)[0]
num_test = int(np.floor(edges.shape[0] / 40.))
num_val = int(np.floor(edges.shape[0] / 40.))

all_edge_idx = list(range(edges.shape[0]))
np.random.shuffle(all_edge_idx)
val_edge_idx = all_edge_idx[:num_val]
test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
test_edges = edges[test_edge_idx]
val_edges = edges[val_edge_idx]
train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

In [13]:
def ismember(a, b, tol=5):
    rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
    return (np.all(np.any(rows_close, axis=-1), axis=-1) and
            np.all(np.any(rows_close, axis=0), axis=0))

In [14]:
len(test_edges)

6445

In [15]:
test_edges_false = []
while len(test_edges_false) < len(test_edges):
    idx_i = np.random.randint(0, adj.shape[0])
    idx_j = np.random.randint(0, adj.shape[0])
    if idx_i == idx_j:
        continue
    if ismember([idx_i, idx_j], edges_all):
        continue
    if test_edges_false:
        if ismember([idx_j, idx_i], np.array(test_edges_false)):
            continue
        if ismember([idx_i, idx_j], np.array(test_edges_false)):
            continue
    test_edges_false.append([idx_i, idx_j])
    print('\r%d / %d' % (len(test_edges_false), len(test_edges)), end='')

6445 / 6445

In [16]:
val_edges_false = []
while len(val_edges_false) < len(val_edges):
    idx_i = np.random.randint(0, adj.shape[0])
    idx_j = np.random.randint(0, adj.shape[0])
    if idx_i == idx_j:
        continue
    if ismember([idx_i, idx_j], train_edges):
        continue
    if ismember([idx_j, idx_i], train_edges):
        continue
    if ismember([idx_i, idx_j], val_edges):
        continue
    if ismember([idx_j, idx_i], val_edges):
        continue
    if val_edges_false:
        if ismember([idx_j, idx_i], np.array(val_edges_false)):
            continue
        if ismember([idx_i, idx_j], np.array(val_edges_false)):
            continue
    val_edges_false.append([idx_i, idx_j])
    print('\r%d / %d' % (len(val_edges_false), len(val_edges)), end='')

6445 / 6445

In [17]:
#assert ~ismember(test_edges_false, edges_all)
#assert ~ismember(val_edges_false, edges_all)
#assert ~ismember(val_edges, train_edges)
#assert ~ismember(test_edges, train_edges)
#assert ~ismember(val_edges, test_edges)

In [18]:
data = np.ones(train_edges.shape[0])
# Re-build adj matrix
adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
adj_train = adj_train + adj_train.T

In [20]:
if FLAGS.features == 0:
    features = sp.identity(features.shape[0])  # featureless

In [21]:
# Some preprocessing
adj_norm = preprocess_graph(adj)

In [22]:
# Define placeholders
placeholders = {
    'features': tf.sparse_placeholder(tf.float32),
    'adj': tf.sparse_placeholder(tf.float32),
    'adj_orig': tf.sparse_placeholder(tf.float32),
    'dropout': tf.placeholder_with_default(0., shape=())
}

In [23]:
num_nodes = adj.shape[0]

In [24]:
features = sparse_to_tuple(features.tocoo())
num_features = features[2][1]
features_nonzero = features[1].shape[0]

In [25]:
# Create model
model = None
if model_str == 'gcn_ae':
    model = GCNModelAE(placeholders, num_features, features_nonzero)
elif model_str == 'gcn_vae':
    model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)

pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

In [26]:
# Optimizer
with tf.name_scope('optimizer'):
    if model_str == 'gcn_ae':
        opt = OptimizerAE(preds=model.reconstructions,
                          labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                      validate_indices=False), [-1]),
                          pos_weight=pos_weight,
                          norm=norm)
    elif model_str == 'gcn_vae':
        opt = OptimizerVAE(preds=model.reconstructions,
                           labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                       validate_indices=False), [-1]),
                           model=model, num_nodes=num_nodes,
                           pos_weight=pos_weight,
                           norm=norm)

In [27]:
# Initialize session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

cost_val = []
acc_val = []

In [28]:
def get_roc_score(edges_pos, edges_neg, emb=None):
    if emb is None:
        feed_dict.update({placeholders['dropout']: 0})
        emb = sess.run(model.z_mean, feed_dict=feed_dict)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [29]:
cost_val = []
acc_val = []
val_roc_score = []

adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})
    # Run single weight update
    outs = sess.run([opt.opt_op, opt.cost, opt.accuracy], feed_dict=feed_dict)

    # Compute average loss
    avg_cost = outs[1]
    avg_accuracy = outs[2]

    roc_curr, ap_curr = get_roc_score(val_edges, val_edges_false)
    val_roc_score.append(roc_curr)

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
          "train_acc=", "{:.5f}".format(avg_accuracy), "val_roc=", "{:.5f}".format(val_roc_score[-1]),
          "val_ap=", "{:.5f}".format(ap_curr),
          "time=", "{:.5f}".format(time.time() - t))

print("Optimization Finished!")

Epoch: 0001 train_loss= 0.63571 train_acc= 0.63658 val_roc= 0.61626 val_ap= 0.58067 time= 0.99052
Epoch: 0002 train_loss= 0.65905 train_acc= 0.63658 val_roc= 0.61574 val_ap= 0.58084 time= 0.83648
Epoch: 0003 train_loss= 0.62683 train_acc= 0.63658 val_roc= 0.61535 val_ap= 0.58081 time= 0.86959
Epoch: 0004 train_loss= 0.67172 train_acc= 0.63658 val_roc= 0.61552 val_ap= 0.58080 time= 0.88174
Epoch: 0005 train_loss= 0.63305 train_acc= 0.63658 val_roc= 0.61572 val_ap= 0.58074 time= 1.05363
Epoch: 0006 train_loss= 0.62703 train_acc= 0.63658 val_roc= 0.61581 val_ap= 0.58074 time= 0.89834
Epoch: 0007 train_loss= 0.63601 train_acc= 0.63658 val_roc= 0.61590 val_ap= 0.58073 time= 0.86383
Epoch: 0008 train_loss= 0.64218 train_acc= 0.63658 val_roc= 0.61585 val_ap= 0.58073 time= 0.87182
Epoch: 0009 train_loss= 0.64298 train_acc= 0.63658 val_roc= 0.61573 val_ap= 0.58076 time= 0.97224
Epoch: 0010 train_loss= 0.63995 train_acc= 0.63658 val_roc= 0.61555 val_ap= 0.58077 time= 0.89961
Epoch: 0011 train_lo

In [30]:
roc_score, ap_score = get_roc_score(test_edges, test_edges_false)
print('Test ROC score: ' + str(roc_score))
print('Test AP score: ' + str(ap_score))

Test ROC score: 0.615566784892
Test AP score: 0.572067873101


In [93]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

edges_pos = test_edges
edges_neg = test_edges_false
feed_dict.update({placeholders['dropout']: 0})
emb = sess.run(model.z_mean, feed_dict=feed_dict)


# Predict on test set of edges
adj_rec = np.dot(emb, emb.T)
preds = []
pos = []
for e in edges_pos:
    preds.append(sigmoid(adj_rec[e[0], e[1]]))
    pos.append(adj_orig[e[0], e[1]])

preds_neg = []
neg = []
for e in edges_neg:
    preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
    neg.append(adj_orig[e[0], e[1]])

preds_all = np.hstack([preds, preds_neg])
labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])

In [99]:
preds_all.shape

(12890,)

In [103]:
len(preds)

6445

In [104]:
len(preds_neg)

6445

In [ ]:
## Essayer de comprendre ce que renvoie preds ! Des proba a thresholder ?
## Je me suis emballée trop vite, c'est pas des 0,1 les prédictions, plutot des proba je crois